## To-do list
- metrics functions
- check for directories structure (model for prediction, databases, saving directory for the drawdown/VaR/etc plots, backup directory for model, backup directory for plots)
- import metrics (VaR, cVaR, drawdown, etc.) functions from metrics.py
- Binance access and etc (copy from Touring)
- Copy keys.py explanation from Touring
- checks in pairs.csv the cryptos from top20_cryptos()


In [ ]:
# ------ Import basic packages
#import matplotlib.pyplot as plt
#import seaborn as sns
#import smtplib  # Needed for the e-mail reports
#import binance.enums  # Responsible for trading

import pandas as pd  # Needed for the formatting below
from keys import api_secret_offline, api_key_offline  # These are your infos, read the README.md if you're lost here
pd.set_option('display.float_format', lambda x: '%.8f' % x)

In [ ]:
# FUNCTIONS

# --- Binance access
def binance_wallet(live_trade=False):
    """
    Description: function that fetches Binance balance for the user

    Inputs: live_trade - bool, default False. If True checks for live trade ability (not needed for backtesting)
    
    Outputs objects: wallet - pd.DataFrame, existing assets and balances
                    cliente - object, binance.client.Client (EXCLUDED FROM return)
                    infos - dict, overall informations about the client (EXCLUDED FROM return)
    """
    # Necessary packages
    import pandas as pd
    from binance.client import Client  # Binance

    def request_wallet():
            # Requests user wallet infos
            print('Fetching wallet balance...')
            wallet = pd.DataFrame(infos['balances'])

            # Gets the 'numerical' informations about the balances
            nums = ['free', 'locked']

            # Transform objs in float
            wallet[nums] = wallet[nums].astype(float)

            # Filter the assets with balance
            mask = wallet[nums][wallet[nums] > 0].dropna(how='all').index
            print('Cleaning wallet from non-positive cryptos...')
            wallet = wallet.iloc[mask]  # keep only assets with positive balance

            # If needed, excludes some cryptos (asset blacklisting)
            black_list = ['NFT', 'SHIB', 'BTTC']
            mask = wallet[wallet['asset'].isin(black_list)].index  # blacklist index
            wallet.drop(mask, axis=0, inplace=True)  # dropping blacklist
            print('Done.')

            print(f'\n--> Please note this account type: {infos['accountType']} <--')

            return wallet
    
    # Get overall info from Binance, using offline (not live) keys
    cliente = Client(api_key_offline, api_secret_offline)

    # Checks for systems online
    if cliente.get_system_status()['msg'] != 'normal':
        print('\n\n!!!! **** WARNING **** !!!!\n')
        print('!!!! BINANCE OFFLINE !!!!\n')
        print('Unable to fetch data\n\n')

    else:
        print('\nBinance on-line. Requesting data.')
        # Fetch user data
        infos = cliente.get_account()

        if live_trade == True:
            # Check if the user is able to live trade (not mandatory for offline)
            if infos['canTrade'] == False:
                print('\nWARNING! User unable to trade, please check status with Binance!')
                print('Aborting.')
            else:
                wallet = request_wallet()

        else:
            wallet = request_wallet()
    
    return wallet


#------ Historical data
def historical_data(ticker='BTCUSDT', days=30, interval='15m'):  # the ticker in Binance works in pairs - here you'll want to know how much is BTC worth in USDT, for example
    """
    Description: gets the trading pair historical data from Binance.

    Inputs: ticker - str, default 'BTCUSDT', the pair you want to trade;
            days - int, default 30, gets historical data from this many days ago;
            interval - str, default '15m', the 'slicing' of the timeframe, more info at https://developers.binance.com/docs/derivatives/coin-margined-futures/market-data/Kline-Candlestick-Data
    
    Outputs objects: hist - pd.DataFrame, OHLC historical data
    """
    import datetime
    import requests
    import json
    import time
    import pandas as pd

    # Defines the data timespan
    end_time = datetime.datetime.now()
    start_time = end_time - datetime.timedelta(days=days)
    
    # Converts time to Unix (because Binance) in miliseconds
    end_timestamp = int(end_time.timestamp()*1000)
    start_timestamp = int(start_time.timestamp()*1000)

    # Binance endpoint
    endpoint = 'https://api.binance.com/api/v3/klines'

    # Timewindow estabilished, requests historical data.
    # Request parameters.
    limit = 1000
    params = {'symbol': ticker, 'interval': interval,
          'endTime': end_timestamp, 'limit': limit,
          'startTime': start_timestamp}
    print('Requesting informations from Binance.')

    # Make the request and saves it in a list. 'Dados' means 'data' in portuguese.
    dados = []
    while True:
        response = requests.get(endpoint, params=params)
        klines = json.loads(response.text)
        dados += klines
        if len(klines) < limit:
            break
        params['startTime'] = int(klines[-1][0])+1
        time.sleep(0.1)
    print('Request successful. Splitting data...')

    # Pick specific data from fetched data
    # About kline[n] pos: https://developers.binance.com/docs/derivatives/coin-margined-futures/market-data/Kline-Candlestick-Data
    loose_data = [[float(kline[1]), float(kline[2]), float(kline[3]), float(kline[4])] for kline in dados]

    # Creates the DataFrame
    hist = pd.DataFrame(loose_data, columns=['open', 'high', 'low', 'close'])
    timestamps = [datetime.datetime.fromtimestamp(int(kline[0])/1000) for kline in dados]
    hist['timestamp'] = timestamps

    # Index as time series
    hist.set_index('timestamp', inplace=True)

    # Registers the trading pair on the DataFrame
    hist['par'] = ticker

    print('All done.')

    return hist


#------ Check for trading pairs CSV file
def check_pairs():
    """
    Description: checks if there's a trading pairs record, makes up the trading pairs from Binance toAsset and fromAsset data if none is found
    Inputs: none
    Outputs: none (generates a csv file though)
    """
    import datetime
    import requests
    import json
    import os
    import pandas as pd

    # Stores current month and year, used to validate files
    today = datetime.datetime.now().strftime('%Y%m')
    
    # Resources and backup path
    resources_dir = './resources/'
    backup_dir = 'older_versions/'

    def create_pairs_file():
        # Sets Binance endpoint and its parameter (crypto common to all trades, in this case)
        pairs_endpoint = 'https://api.binance.com/sapi/v1/convert/exchangeInfo'
        params = {'toAsset': 'USDT'}

        # Makes the request
        response = requests.get(pairs_endpoint, params=params)
        
        # Changes the response into a DataFrame and creates the buy/sell pairs
        df = pd.DataFrame(json.loads(response.text))
        sell_pairs = df['fromAsset'] + df['toAsset']
        buy_pairs = df['toAsset'] + df['fromAsset']

        # Creates a new temp DataFrame with just the pairs
        temp = pd.concat([buy_pairs, sell_pairs], axis=1)
        temp.columns = ['Buy Pairs', 'Sell Pairs']

        # Saves to file
        temp.to_csv(f'{resources_dir}pairs_{today}.csv', index=False)


    # Checks for path
    if not os.path.exists(resources_dir):
        print(f"Directory '{resources_dir}' does not exist. Let's make it, shall we?")
        os.mkdir(resources_dir)
        print(f"Done, directory '{resources_dir}' created successfully.")
    
    else:
        print('The resources directory exists, checking for trade pairs file.')

    # Check for any pair file. If it exists and is newer than a month, loads it.
    arqs = os.listdir(resources_dir)

    # Keeps only generated files, disregarding folders or other misc files
    for i in arqs:
        if 'pairs' not in str(i):
            arqs.remove(i)

    if len(arqs) == 0:
        print('Trading file not found in folder. Creating...')
        create_pairs_file()
        print(f"Trading pairs file created: 'pairs_{today}.csv'")

    elif len(arqs) == 1:
        arqs = arqs[0]
        print(f"Trading pairs file '{arqs}' found, checking version.")
        file_found = arqs.split('.csv')[0]
        file_found = file_found.split('_')[1]

        if int(today) > int(file_found):
            import shutil                        
            print(f"Time to update the files! Moving current file to './{backup_dir}'.")
            shutil.move(resources_dir+arqs, resources_dir+backup_dir+arqs)
            print(f'Updating trading pairs file.')
            create_pairs_file()
            print(f"New trading pairs file created: 'pairs_{today}.csv'")
            print('All done.')

        else:
            print('Trading pairs file is up to date.')


    elif len(arqs) > 1:
        print(f"WARNING: Multiple files found! Moving all trading pairs files to './{backup_dir}'.")
        for i in arqs:
            import shutil
            shutil.move(resources_dir+i, resources_dir+backup_dir+i)
        print(f'Creating valid trading pairs file.')
        create_pairs_file()
        print(f"New trading pairs file created: 'pairs_{today}.csv'")
        print('All done.')
    
    else:
        print("Some kind of witchcraft error happened. This message isn't supposed to show up!")


#------ List with the 20 top market cap currencies
def top20_cryptos():
    """
    Description: get the 20 biggest market cap cryptos from the web. Needs tweaks for each source.

    input: none

    outpu: list, cryptocurrencies symbols
    """
    import requests
    from bs4 import BeautifulSoup

    cmc = 'https://crypto.com/price'
    
    try:
        response = requests.get(cmc)
        soup = BeautifulSoup(response.text, "html.parser")

        site = soup.find_all("span", {"class": "chakra-text"})

        cryptos = []
        for cur in site:
            cryptos.append(cur.get_text())
    
    except:
        print(f"Error fetching biggest market cap cryptos from {cmc}")
    
    return cryptos[0:20]




In [ ]:
crypto_list = top20_cryptos()

In [255]:
crypto_list

['BTC',
 'ETH',
 'USDT',
 'XRP',
 'BNB',
 'SOL',
 'USDC',
 'TRX',
 'DOGE',
 'ADA',
 'WBTC',
 'BCH',
 'SUI',
 'LINK',
 'LEO',
 'AVAX',
 'XLM',
 'TON',
 'SHIB',
 'WBT']

In [37]:
#from metrics import mvcriterion, optim_mvcrit

#wallet = binance_wallet()

#print('\nThis is the wallet obj:')
#print(wallet)

#hist = historical_data()
#print(hist)

#teste = ['BTCUSDT', 'BTCIOTA']

#historical_data(ticker='BTCIOTA')

infos

{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00100000',
  'taker': '0.00100000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'preventSor': False,
 'updateTime': 1744356024027,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00000691', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locke